### Uso de las políticas de archivo de S3 Intelligent-Tiering para archivar automáticamente los objetos de S3

#### Problema
Necesita realizar una transición automática de los objetos a los que se accede con poca frecuencia a una clase de almacenamiento de archivo diferente sin que ello afecte al rendimiento o añada una sobrecarga operativa.

#### Solución
Cree una política para automatizar el archivado de los objetos de S3 en el archivo de S3 Glacier en función de los patrones de acceso a los objetos con más de 90 días de antigüedad, luego aplíquela a su bucket de S3.

In [1]:
import boto3
import json
import random
from pprint import pprint 

region_aws = 'us-east-1'

ec2 = boto3.resource('ec2', region_name=region_aws)
ec2_client = boto3.client('ec2', region_name=region_aws)
s3 = boto3.resource('s3')
s3_client = boto3.client('s3')

In [2]:
# Create a bucket
bucket_name = 'cookbook-{}'.format(random.randint(10000, 1000000))
bucket = s3.create_bucket(Bucket=bucket_name)
print("Bucket name: {}".format(bucket_name))

Bucket name: cookbook-943240


Por defecto cuando creamos un bucket, las opciones de **Block all public access** (Bloquear todo el acceso público) están desactivadas por defecto.

In [3]:
response = s3_client.put_public_access_block(
        Bucket=bucket_name,
        PublicAccessBlockConfiguration={
            'BlockPublicAcls': True,
            'IgnorePublicAcls': True,
            'BlockPublicPolicy': True,
            'RestrictPublicBuckets': True
        }
)

In [4]:
# Create policy
policy_bucket = {
  "Id": "awscookbook302",
  "Status": "Enabled",
  "Tierings": [
    {
      "Days": 90,
      "AccessTier": "ARCHIVE_ACCESS"
    }
  ]
}

In [5]:
# Apply the Intelligent-Tiering configuration:
response = s3_client.put_bucket_intelligent_tiering_configuration(
    Bucket=bucket_name,
    Id='awscookbook302',
    IntelligentTieringConfiguration=policy_bucket
)

Asegúrese de que su caso de uso y sus aplicaciones pueden soportar el mayor tiempo de recuperación asociado al nivel de almacenamiento de S3 Glacier. Puede configurar su aplicación para que utilice un mecanismo de recuperación acelerada compatible con el archivo de S3 Glacier para reducir el tiempo de recuperación pero aumentar el coste. Para obtener una lista completa de los tiempos de archivo y cómo configurar el acceso acelerado, consulte el [documento de soporte](https://oreil.ly/tru0v).

#### Controles de validación

In [6]:
# Get the configuration of Intelligent-Tiering for your bucket:
response = s3_client.get_bucket_intelligent_tiering_configuration(
    Bucket=bucket_name,
    Id='awscookbook302'
)

In [12]:
pprint(response['IntelligentTieringConfiguration'])

{'Id': 'awscookbook302',
 'Status': 'Enabled',
 'Tierings': [{'AccessTier': 'ARCHIVE_ACCESS', 'Days': 90}]}


In [14]:
# Upload an object to the bucket:
s3_client.upload_file('new-report.png', bucket_name, 'new-report.png')

In [18]:
# Check the storage class for the object:
response = s3_client.list_objects_v2(
    Bucket=bucket_name
)

pprint(response)

{'Contents': [{'ETag': '"75acf5a0dd2f6bdd67c36fa2748a1a19"',
               'Key': 'new-report.png',
               'LastModified': datetime.datetime(2022, 10, 16, 14, 15, 47, tzinfo=tzutc()),
               'Size': 86065,
               'StorageClass': 'STANDARD'}],
 'EncodingType': 'url',
 'IsTruncated': False,
 'KeyCount': 1,
 'MaxKeys': 1000,
 'Name': 'cookbook-943240',
 'Prefix': '',
 'ResponseMetadata': {'HTTPHeaders': {'content-type': 'application/xml',
                                      'date': 'Sun, 16 Oct 2022 14:18:28 GMT',
                                      'server': 'AmazonS3',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bucket-region': 'us-east-1',
                                      'x-amz-id-2': 'QZT+FuKsWpgH2cF/fgL8X9vyFYfFiNmA8VZ/QVk490t52NKLbOXAQucHPxF9AGLBTZUxijviDT8=',
                                      'x-amz-request-id': 'Z4SEKD56G4V04YQ5'},
                      'HTTPStatusCode': 2

### Discusión
Un archivo S3 Intelligent-Tiering proporciona un mecanismo automático para la transición de objetos "fríos" (a los que se accede con menos frecuencia) a un archivo S3 Glacier. Puede definir el tiempo necesario para la transición de un objeto al archivo (entre 90 y 730 días). Esta función ayuda a cumplir con los requisitos de retención a largo plazo que puede tener para el cumplimiento. Los niveles de almacenamiento disponibles en S3 Intelligent-Tiering se asignan directamente a los niveles de S3:

* **Frequent Access** - Optimizado para el acceso frecuente (S3 Standard)
* **Infrequent Access** - Optimizado para el acceso poco frecuente (S3 Standard-IA)
* **Archive Access** - Para fines de archivo (S3 Glacier)
* **Deep Archive Access** - Propósitos de retención a largo plazo (S3 Glacier Deep Archive)

Esta configuración de archivo está separada de la configuración principal del nivel de S3 Intelligent-Tiering que se coloca en los objetos, ya que se trata de una configuración específica del bucket. En la receta anterior, usted configuró una regla de ciclo de vida para configurar todos los objetos dentro de un bucket para la transición al nivel de almacenamiento de S3 Intelligent-Tiering. Esta receta añade una configuración adicional para realizar la transición de los objetos a los niveles de almacenamiento de S3 en función de su configuración. Puede utilizar cualquiera de estos métodos por separado o ambos simultáneamente para satisfacer sus propios requisitos.

**NOTA:** Los niveles de S3 son específicos del objeto, lo que difiere de que el archivo de Intelligent-Tiering sea específico del cubo. Puede filtrar una configuración de archivo para que se aplique sólo a determinados prefijos, etiquetas de objetos y nombres de objetos si desea incluir o excluir objetos en una configuración. Para más información, consulte el documento de soporte.

